In [ ]:
import xarray as xr
import json

In [ ]:
# merge all "x_leadtimes_allmodels.nc" together

In [ ]:
# Open the datasets
r = xr.open_dataset("PearsonR_leadtimes_allmodels.nc")
rmse = xr.open_dataset("RMSE_leadtimes_allmodels.nc")
mae = xr.open_dataset("MAE_leadtimes_allmodels.nc")
mbe = xr.open_dataset("MBE_leadtimes_allmodels.nc")

# List of shared variables
variables = list(set(r.data_vars) & set(rmse.data_vars) & set(mae.data_vars) & set(mbe.data_vars))

# Build a new dataset with a 'metric' dimension
combined_vars = {}
for var in variables:
    combined_vars[var] = xr.concat(
        [r[var].expand_dims(metric=["r"]),
         rmse[var].expand_dims(metric=["rmse"]),
         mae[var].expand_dims(metric=["mae"]),
         mbe[var].expand_dims(metric=["mbe"]),
        ],
        dim="metric"
    )

# Combine all variables into one Dataset
combined = xr.Dataset(combined_vars)

combined.to_netcdf("R_RMSE_MAE_MBE_leadtimes_allmodels.nc")

# Final structure
print(combined)

In [ ]:
# turn the "R_RMSE_MAE_MBE_leadtimes_allmodels.nc" into a json file

In [ ]:
# Load dataset
ds = xr.open_dataset("R_RMSE_MAE_MBE_leadtimes_allmodels.nc")
# Drop the 'number' coordinate if it exists
if "number" in ds.coords:
    ds = ds.drop_vars("number")

# Convert to xarray.Dataset → pandas.DataFrame
df = ds.to_dataframe().reset_index()


# Convert DataFrame to a list of dictionaries (good for plotting)
data = df.to_dict(orient="records")

# Save to JSON
with open("R_RMSE_MAE_MBE_leadtimes_allmodels.json", "w") as f:
    json.dump(data, f, indent=2)

print("Saved as R_RMSE_MAE_MBE_leadtimes_allmodels.json")